In [ ]:
#"Decomment for install"
#!pip install kmeans
#!pip install seaborn

In [2]:
#Modules to install via pip pandas,ipynb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pprint import pprint
import os
import import_ipynb
import sys
import kmeans
sys.path.append('../')
from functions import *
from pandas.plotting import scatter_matrix
import cmath as math
from mpl_toolkits.mplot3d import Axes3D
from sklearn import cluster
from sklearn.metrics import confusion_matrix
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.cluster import KMeans
# scipy
from scipy.cluster.vq import kmeans,vq,whiten
import sklearn.metrics as sm
import seaborn as sns
from sklearn.decomposition import PCA
%matplotlib inline
import random
random.seed(6666)

In [5]:

dataList=coojaJsonImporter("./traces")
data=[]
cases=[
      "BH1",
        "BH2",
    "normal"
      ]

BlackHole=[-1,4,5+8]
for nodeList in dataList:
    data.append(createNodes(nodeList))

#All data collection is in variable node that is a list of list of nodes
#3 nets input x 9 nodes by net
data[0][0].pkts[1:5] 
columns=[
    "count",
    "label"
]
d={ "label":[],
    "count":[],
    "std":  [],
    "mean": [],
    "var":  [],
    "25%":  [],
   "50%":[],
   "75%":[],
   "min":[],
   "max":[],
   "hop":[]
   #"rtt/pkt":[]
}
count=[]
labels=[]
var=[]
#stats=pd.DataFrame(columns=columns)
rows=[count,labels]
for i in range(len(data)):
    for j in range(len(data[i])):
            #df1 = df1.assign(e=p.Series(np.random.randn(sLength)).values)
            node=data[i][j]
            name=str(j)+" "+cases[i]
            #print(name)
            #print(node.pkts['pkt'].count())
            #stats["count"]=node.pkts['pkt'].count()
            d["count"].append(data[i][j].pkts['rtt'].count())
            d["std"].append(data[i][j].pkts['rtt'].std())
            d["mean"].append(data[i][j].pkts['rtt'].mean())
            d["var"].append(data[i][j].pkts['rtt'].var())
            d["label"].append(cases[i])
            d["25%"].append(data[i][j].pkts['rtt'].describe()["25%"])
            d["50%"].append(data[i][j].pkts['rtt'].describe()["50%"])
            d["75%"].append(data[i][j].pkts['rtt'].describe()["75%"])
            d["min"].append(data[i][j].pkts['rtt'].describe()["min"])
            d["max"].append(data[i][j].pkts['rtt'].describe()["max"])
            d["hop"].append(float(data[i][j].hop))
            ## d["max"].append(data[i][j].pkts['rtt']
            #var.append(data[i][j].pkts['pkt'].var())
            #stats[name]
            #stats= stats.assign(ip=pd.Series(node.pkts['pkt'].describe()).values)
            #stats.append(node.pkts['pkt'].describe())

            
            
stats=pd.DataFrame(d)            

stats
correction=[]
correction2=[] #for 3 cluster 0:normal net 1:bh net 2:bh
correction3=[] #for 3 cluster 0:normal net 1:bh net 2:bh net
for i in range(len(d["label"])):
    el=d["label"][i]
    if el=="normal":
        correction.append(1)
        correction2.append(1)
        
    else:
        #print(el=="BH2" and i==BlackHole[2])
        correction.append(0)
        if ((el=="BH1" and i==BlackHole[1]) or
            (el=="BH2" and i==BlackHole[2])):
            
            correction2.append(2)
        else:
            correction2.append(0)
        

correction=np.array(correction)
correction2=np.array(correction2)
print(correction2)
stats.head()
#findMissingPackets(data[0][0])
print(data[0][0].pktsC)

Importing test_1BH_2018-11-09_12_31_25.json
Importing test_1BH_2018-11-09_14_37_46.json
Importing test_nom_2018-11-09_08_55_11.json
[0 0 0 0 2 0 0 0 0 0 0 0 0 2 0 0 0 0 1 1 1 1 1 1 1 1 1]
     pkt     rtt   ttl
0    0.0  1342.0  63.0
1    1.0  8170.0  63.0
2    2.0   973.0  63.0
3    3.0   848.0  63.0
4    4.0  1022.0  63.0
5    5.0  2406.0  63.0
6    6.0   849.0  63.0
7    7.0  3766.0  63.0
8    8.0  1406.0  63.0
9    9.0   824.0  63.0
10  10.0   862.0  63.0
11  11.0  3338.0  63.0
12  12.0  2104.0  63.0
13  13.0  1001.0  63.0
14  14.0  2940.0  63.0
15  15.0   904.0  63.0
16  16.0   976.0  63.0
17  17.0  1321.0  63.0
18  18.0  1484.0  63.0
19  19.0  1790.0  63.0
20  20.0  1300.0  63.0
21  21.0  1213.0  63.0
22  22.0   746.0  63.0
23  23.0   818.0  63.0
24  24.0  2494.0  63.0
25  25.0   709.0  63.0
26  26.0  1031.0  63.0
27  27.0  1623.0  63.0
28  28.0  1018.0  63.0
29  29.0  1406.0  63.0
..   ...     ...   ...
68  70.0  1403.0  63.0
69  71.0  1052.0  63.0
70  72.0  1362.0  63.0
71  73.

In [ ]:
data=stats.drop(["label","25%","std","mean","75%","50%","min","max"],axis=1)

dataC=stats["label"]
print(data.head())
#Y = data[['var']]
#X = data[['std']]



Kmeans with 2 clusters, using just count, mean and hop feature we can get 85% of understanding probability


In [ ]:

kmeans = KMeans(n_clusters=2)
kmeans.fit(data)
labels = kmeans.predict(data)
centroids = kmeans.cluster_centers_


print(labels)
print(correction)

# Performance Metrics
accuracy=sm.accuracy_score(correction, labels)

# Confusion Matrix
confusionMatrix=sm.confusion_matrix(correction, labels)

print(accuracy)
print(confusionMatrix)

In [ ]:
dataplus=stats.drop(["label"],axis=1)
data=dataplus
kmeans = KMeans(n_clusters=2)
kmeans.fit(data)
labels = kmeans.predict(data)
centroids = kmeans.cluster_centers_


print(labels)
print(correction)

# Performance Metrics
accuracy=sm.accuracy_score(correction, labels)

# Confusion Matrix
confusionMatrix=sm.confusion_matrix(correction, labels)

print(accuracy)
print(confusionMatrix)




In [ ]:
Y=data
Nc = range(1, 20)

kmeans = [KMeans(n_clusters=i) for i in Nc]

kmeans

score = [kmeans[i].fit(Y).score(Y) for i in range(len(kmeans))]

score

plt.plot(Nc,score)

plt.xlabel('Number of Clusters')

plt.ylabel('Score')

plt.title('Elbow Curve')

plt.show()


In [ ]:
fig = plt.figure(figsize=(5, 5))

colors = map(lambda x: colmap[x+1], labels)


plt.scatter(data['count'], data['var'], alpha=0.5)
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=50)

plt.show()

In [ ]:
#https://www.learndatasci.com/tutorials/k-means-clustering-algorithms-python-intro/
#df=np.array(X,Y)




In [ ]:


X = dataplus.iloc[:,0:4]
pca = PCA(n_components=2)
pca.fit(X)
X_ = pca.transform(X)

dfPCA = pd.DataFrame({'x1': X_[:,0], 'x2': X_[:,1]})
dfPCA['labels'] = stats['label']
dfPCA.head()

In [ ]:

labels = stats['label'].unique().tolist()
plt.figure(figsize=(7,5))
for lab in labels:
    plt.scatter(dfPCA.loc[dfPCA['labels'] == lab, 'x1'],  dfPCA.loc[dfPCA['labels'] == lab, 'x2'], label=lab)
    plt.legend()

Experiment with 3 clusters trying also to find the black Hole node, 0.74% of probability finding a black hole in the network

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(dataplus)
labels = kmeans.predict(dataplus)
centroids = kmeans.cluster_centers_


print(labels)
print(correction2)


# Performance Metrics
accuracy=sm.accuracy_score(correction2, labels)

# Confusion Matrix
confusionMatrix=sm.confusion_matrix(correction2, labels)

print(accuracy)
print(confusionMatrix)

In [ ]:
for i in range(len(labels)):
    if labels[i]==2:
        labels[i]=0
       
    if correction2[i]==2:
        correction2[i]=0
print(labels)
print(correction2)        
# Performance Metrics
accuracy=sm.accuracy_score(correction2, labels)

# Confusion Matrix
confusionMatrix=sm.confusion_matrix(correction2, labels)

print(accuracy)
print(confusionMatrix)

Example adding also the 16 nodes, clearly the accuracy goes down as we dont have many data

In [ ]:
dataList=coojaJsonImporter("../cooja-16nodes/traces/")

data=[]
cases=[
      "BH1",
        "BH2",
    "normal",
    "BH3"
      ]

BlackHole=[-1,4,5+8,-1]
for nodeList in dataList:
    data.append(createNodes(nodeList))
dataList=coojaJsonImporter("../cooja-9nodes/traces/")
for nodeList in dataList:
    data.append(createNodes(nodeList))
    
d={ "label":[],
    "count":[],
    "std":  [],
    "mean": [],
    "var":  [],
    "25%":  [],
   "50%":[],
   "75%":[],
   "min":[],
   "max":[]
   #"rtt/pkt":[]
}
count=[]
labels=[]
var=[]
#stats=pd.DataFrame(columns=columns)
rows=[count,labels]
for i in range(len(data)):
    for j in range(len(data[i])):
            #df1 = df1.assign(e=p.Series(np.random.randn(sLength)).values)
            node=data[i][j]
            name=str(j)+" "+cases[i]
            d["count"].append(data[i][j].pkts['rtt'].count())
            d["std"].append(data[i][j].pkts['rtt'].std())
            d["mean"].append(data[i][j].pkts['rtt'].mean())
            d["var"].append(data[i][j].pkts['rtt'].var())
            d["label"].append(cases[i])
            d["25%"].append(data[i][j].pkts['rtt'].describe()["25%"])
            d["50%"].append(data[i][j].pkts['rtt'].describe()["50%"])
            d["75%"].append(data[i][j].pkts['rtt'].describe()["75%"])
            d["min"].append(data[i][j].pkts['rtt'].describe()["min"])
            d["max"].append(data[i][j].pkts['rtt'].describe()["max"])
          
            
            
stats=pd.DataFrame(d)        
correction=[]
correction2=[] #for 3 cluster 0:normal net 1:bh net 2:bh
for i in range(len(d["label"])):
    el=d["label"][i]
    if el=="normal":
        correction.append(1)
        correction2.append(1)
        
    else:
        #print(el=="BH2" and i==BlackHole[2])
        correction.append(0)
        if ((el=="BH1" and i==BlackHole[1]) or
            (el=="BH2" and i==BlackHole[2])):
            
            correction2.append(2)
        else:
            correction2.append(0)
        

correction=np.array(correction)
correction2=np.array(correction2)
data=stats.drop(["label","25%","var","75%","50%","min","max","mean"],axis=1)

kmeans = KMeans(n_clusters=2)
kmeans.fit(data)
labels = kmeans.predict(data)
centroids = kmeans.cluster_centers_


print(labels)
print(correction)

# Performance Metrics
accuracy=sm.accuracy_score(correction, labels)

# Confusion Matrix
confusionMatrix=sm.confusion_matrix(correction, labels)

print(accuracy)
print(confusionMatrix)



#Last experiment

using windows so we can have virtually more data to work



In [ ]:

dataList=coojaJsonImporter("./traces")
data=[]
cases=[
      "BH1",
        "BH2",
    "normal"
      ]

for nodeList in dataList:
    data.append(createNodes(nodeList))
    
d={ "label":[],
    "count":[],
    "std":  [],
    "mean": [],
    "var":  [],
    "25%":  [],
    "50%":[],
   "75%":[],
   "min":[],
   "max":[],
   "hop":[],
   "missing packets":[]
   #"rtt/pkt":[]
}

window=25
for i in range(len(data)):
    for j in range(len(data[i])):
        n=len(data[i][j].pkts["rtt"])
        #print(n)
        for z in range(0,n,window):
            #print(z)
            
            #df1 = df1.assign(e=p.Series(np.random.randn(sLength)).values)
            node=data[i][j]
            name=str(j)+" "+cases[i]
            d["count"].append(data[i][j].pkts['rtt'][z:z+window].count())
            d["std"].append(data[i][j].pkts['rtt'][z:z+window].std())
            d["mean"].append(data[i][j].pkts['rtt'][z:z+window].mean())
            d["var"].append(data[i][j].pkts['rtt'][z:z+window].var())
            d["label"].append(cases[i])
            d["25%"].append(data[i][j].pkts['rtt'][z:z+window].describe()["25%"])
            d["50%"].append(data[i][j].pkts['rtt'][z:z+window].describe()["50%"])
            d["75%"].append(data[i][j].pkts['rtt'][z:z+window].describe()["75%"])
            d["min"].append(data[i][j].pkts['rtt'][z:z+window].describe()["min"])
            d["max"].append(data[i][j].pkts['rtt'][z:z+window].describe()["max"])
            d["hop"].append(data[i][j].hop)
            missingPackets=data[i][j].pkts['rtt'][z:z+window]


In [ ]:
for i in range(len(data)):
    for j in range(len(data[i])):
        n=len(data[i][j].pkts["rtt"])
        #print(n)

        
#print(data[0][0].pkts["pkt"])        

In [ ]:
stats=pd.DataFrame(d)            
print(stats.shape)
stats=stats.dropna()
print(stats.columns)

In [ ]:

data=stats.drop(["label","25%","count","75%","50%","min","max","mean"],axis=1)

#dataplus=stats.drop(["label"],axis=1)

#data=dataplus
kmeans = KMeans(n_clusters=2)
kmeans.fit(data)
labels = kmeans.predict(data)
centroids = kmeans.cluster_centers_

data.columns


In [ ]:
correction=[]
#stats=stats.dropna()

col=np.array(stats["label"])

for i in range(len(col)):
    el=col[i]
    if el=="normal":
        correction.append(1)
      
        
    else:
       
        correction.append(0)
        

correction=np.array(correction)
print(len(correction))
print(len(labels))

In [ ]:
#print(labels)
#print(correction)

# Performance Metrics
accuracy=sm.accuracy_score(correction, labels)

# Confusion Matrix
confusionMatrix=sm.confusion_matrix(correction, labels)

print(correction)
print(labels)
print(accuracy)
print(confusionMatrix)

In [ ]:
X = data.iloc[:,0:4]
pca = PCA(n_components=2)
pca.fit(X)
X_ = pca.transform(X)

dfPCA = pd.DataFrame({'x1': X_[:,0], 'x2': X_[:,1]})
dfPCA['labels'] = stats['label']
dfPCA.head()



In [ ]:
labels = stats['label'].unique().tolist()
plt.figure(figsize=(7,5))
for lab in labels:
    plt.scatter(dfPCA.loc[dfPCA['labels'] == lab, 'x1'],  dfPCA.loc[dfPCA['labels'] == lab, 'x2'], label=lab, alpha=0.5)
    plt.legend()

In [ ]:


def findMissingPackets(node):
    #print(node.pkts["pkt"])
    maxP=-1
    
    for el in node.pkts["pkt"]:
        if(el>maxP): maxP=int(el)
    #print(maxP)
    pkt=[None]*(maxP+1)
    for i in range(len(node.pkts["pkt"])):
        index=int(node.pkts["pkt"][i])
        #print(index)
        pkt[index]=node.pkts["rtt"][i]
        #pkt[)]=node.pkts["pkt"][i]
    return pkt
      

In [ ]:
print(type(node.pkts["pkt"]))
findMissingPackets(data[0][5])

In [5]:
print(type(pfna))

NameError: name 'na' is not defined

In [17]:
#!/usr/bin/env python
# coding: utf-8

# Parsing json data 
# CoojaJsonImporter(dir) imports jsons from the dir in a list
# dict2df(dict) imports one json and return a df 
# dict2df_list(dict) import one json and return a list of df

# In[9]:


import os
import pandas as pd
import numpy as np
import json

#Object idea= List of nodes
#Node has (ip,hop,min_rtt,max_rtt,pkts,responses)
#pkt is a dataframe with packets
#hop is from 64-ttl


class node(object):
    ip = ""
    hop= 0
    min_rtt= 0
    max_rtt= 0
    pkts=pd.DataFrame()
    pktsC=pd.DataFrame()
    responses=0
    
    
    # The class "constructor" - It's actually an initializer 
    def __init__(self,ip,hop,min_rtt,max_rtt,pkts,responses,pktsC):
        self.ip = ip
        self.hop=hop
        self.min_rtt=min_rtt
        self.max_rtt=max_rtt
        self.pkts=pkts
        self.responses=responses
        self.pktsC=pktsC

    def make_node(ip,hop,min_rtt,max_rtt,pkts,responses,pktsC):
        node= node(ip,hop,min_rtt,max_rtt,pkts,responses,pktsC)
        return node
    
    
class packet(object):
    rtt=np.NaN
    pkt=np.NaN
    ttl=np.NaN   
    def __init__(self,rtt,pkt,ttl):
        self.rtt=rtt
        self.pkt=pkt
        self.ttl=ttl
    
    
    def make_packet(rtt,pkt,ttl):
        package=package(rtt,pkt,ttl)

def coojaJsonImporter(dir):
        
        dataList=[]

        for file in os.listdir(dir):
            print("Importing "+ file)
            with open(dir+"/" + file, 'r') as f:

                dataList.append(json.load(f))

        return dataList

def dict2df(dict):
    dfList=[]
    bigdata=pd.DataFrame()
    for key in dict.keys():
        df=pd.DataFrame(dict[key]['pkts'])
        df['ip']=key
        
        #dfList.append(df)
        #print(df)
        bigdata=bigdata.append(df,ignore_index=True)

    return bigdata
        
def dict2df_list(dict):
    dfList=[]
    #dfList(pd.DataFrame(dict))
    for key in dict.keys():
        df=pd.DataFrame(dict[key]['pkts'])
        df['ip']=key
        df['hop']=64-(df['ttl'])
        dfList.append(df)
        #print(df)
    return dfList


###Function to create nodes, create a list of nodes
### 
    
def createNodes(dict):
    nodeList=[]
    #dfList(pd.DataFrame(dict))
    for ip in dict.keys():
        pkts=pd.DataFrame(dict[ip]['pkts'])
        #(ip,hop,min_rtt,max_rtt,pkts,responses)
        #print(dict.get(ip).get("max_rtt"))
        #findMissingPackets(dict.get(ip))
        pkts1=dict.get(ip).get("pkts")
        pktsList=[]
        for p in pkts1:
            #print(p.get("rtt"))
             #make_packet(rtt,pkt,ttl)
            rtt=p.get("rtt")
            pkt=p.get("pkt")
            ttl=p.get("ttl")
            pack=packet(rtt,pkt,ttl)
            pktsList.append(pack)
        min_rtt=dict.get(ip).get("min_rtt")
        max_rtt=dict.get(ip).get("max_rtt")
        responses=dict.get(ip).get("responses")
        hop=64-(int(pkts[0:1]["ttl"]))
        #print(type(pkts[0:1]["ttl"]))
        #print(pkts[0:1]["ttl"])
        n=node(ip,hop,min_rtt,max_rtt,pkts,responses,pkts)
        
        nodeList.append(n)
        #print(type(nodeList[0].pkts[0]))
        
    return nodeList

In [20]:

dataList=coojaJsonImporter("./traces")
data=[]
cases=[
      "BH1",
        "BH2",
    "normal"
      ]

for nodeList in dataList:
    data.append(createNodes(nodeList))
    
#print(data[0][5].responses)
nodeList["aaaa::212:7404:4:404:"]['pkts']

Importing test_1BH_2018-11-09_12_31_25.json
Importing test_1BH_2018-11-09_14_37_46.json
Importing test_nom_2018-11-09_08_55_11.json


[{'rtt': 862.0, 'pkt': 0.0, 'ttl': 63.0},
 {'rtt': 1188.0, 'pkt': 1.0, 'ttl': 63.0},
 {'rtt': 1333.0, 'pkt': 2.0, 'ttl': 63.0},
 {'rtt': 913.0, 'pkt': 3.0, 'ttl': 63.0},
 {'rtt': 1032.0, 'pkt': 4.0, 'ttl': 63.0},
 {'rtt': 1534.0, 'pkt': 5.0, 'ttl': 63.0},
 {'rtt': 1409.0, 'pkt': 6.0, 'ttl': 63.0},
 {'rtt': 1112.0, 'pkt': 7.0, 'ttl': 63.0},
 {'rtt': 1743.0, 'pkt': 8.0, 'ttl': 63.0},
 {'rtt': 2830.0, 'pkt': 9.0, 'ttl': 63.0},
 {'rtt': 865.0, 'pkt': 10.0, 'ttl': 63.0},
 {'rtt': 731.0, 'pkt': 11.0, 'ttl': 63.0},
 {'rtt': 712.0, 'pkt': 12.0, 'ttl': 63.0},
 {'rtt': 1483.0, 'pkt': 13.0, 'ttl': 63.0},
 {'rtt': 953.0, 'pkt': 14.0, 'ttl': 63.0},
 {'rtt': 1556.0, 'pkt': 15.0, 'ttl': 63.0},
 {'rtt': 1337.0, 'pkt': 16.0, 'ttl': 63.0},
 {'rtt': 1156.0, 'pkt': 17.0, 'ttl': 63.0},
 {'rtt': 968.0, 'pkt': 18.0, 'ttl': 63.0},
 {'rtt': 1217.0, 'pkt': 19.0, 'ttl': 63.0},
 {'rtt': 883.0, 'pkt': 20.0, 'ttl': 63.0},
 {'rtt': 1325.0, 'pkt': 21.0, 'ttl': 63.0},
 {'rtt': 1227.0, 'pkt': 22.0, 'ttl': 63.0},
 {'rtt